## bifacial_radiance examples
what to do with the new bifacial_radiance python module

### Prerequisites:
This software requires the previous installation of RADIANCE from https://github.com/NREL/Radiance/releases

If you are on a PC you should also install the Jaloxa radwinexe executables: http://www.jaloxa.eu/resources/radiance/radwinexe.shtml

Gencumulativesky requires gencumulativesky.exe to be installed in your Radiance directory

In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')

In [2]:
# Simple example system using Radiance.
testfolder = r'C:\Users\cdeline\Documents\Python Scripts\TestFolder'  #point to an empty directory or existing Radiance directory
demo = RadianceObj('simple_panel',testfolder)  # Create a RadianceObj 'object'
demo.setGround(0.62) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
try:
    epwfile = demo.getEPW(37.5,-77.6) #can't run this within NREL firewall. Otherwise, pull TMY data for any global lat/lon
except:
    pass
    
metdata = demo.readEPW('EPWs\\USA_VA_Richmond.Intl.AP.724010_TMY.epw') # read in the weather data
# Now we either choose a single time point, or use cumulativesky for the entire year. 
demo.gendaylit(metdata,4020)  # Noon, June 17th
#demo.genCumSky(demo.epwfile) # entire year.
# create a scene using panels in landscape at 10 deg tilt, 1.5m pitch. 0.2 m ground clearance
sceneDict = {'tilt':10,'pitch':1.5,'height':0.2,'orientation':'landscape','azimuth':180}  
scene = demo.makeScene('simple_panel',sceneDict) #makeScene creates a .rad file with 10 modules per row, 3 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.
analysis = AnalysisObj(octfile, demo.basename)  # return an analysis object including the scan dimensions for back irradiance
analysis.analysis(octfile, demo.basename, scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio: %0.3f - %0.3f' %(min(analysis.backRatio), np.mean(analysis.backRatio)) )



path = C:\Users\cdeline\Documents\Python Scripts\TestFolder
Making path: images/
Making path: objects/
Making path: results/
Making path: skies/
Making path: materials/
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
incorrect panel type selection
created simple_panel.oct linescan in process: simple_panel_Front
linescan in process: simple_panel_Back
saved: results\irr_simple_panel.csv
Annual bifacial ratio: 0.089 - 0.131


In [3]:
# Make a color render and falsecolor image of the scene
analysis.makeImage('side.vp')
analysis.makeFalseColor('side.vp')

generating visible render of scene
generating scene in WM-2
saving scene in false color


In [ ]:
'''
    demo = RadianceObj('G173gencumsky_3.0m')  
    demo.setGround('litesoil')
    metdata = demo.readEPW(r'USA_CO_Boulder.724699_TMY2.epw')
    # sky data for index 4010 - 4028 (June 17)  
    #demo.gendaylit(metdata,4020)
    demo.genCumSky(r'USA_CO_Boulder.724699_TMY2.epw')
    octfile = demo.makeOct(demo.filelist + ['objects\\monopanel_G173_ht_3.0.rad'])
    analysis = AnalysisObj(octfile, demo.basename)
    analysis.G173analysis(octfile, demo.basename)
    
    demo2 = RadianceObj('G173gendaylit_3.0m')  
    demo2.setGround('litesoil')
    metdata = demo2.readEPW(r'USA_CO_Boulder.724699_TMY2.epw')
    # sky data for index 4010 - 4028 (June 17)  
    demo2.gendaylit(metdata,4020)
    #demo.genCumSky(r'USA_CO_Boulder.724699_TMY2.epw')
    octfile = demo2.makeOct(demo.filelist + ['objects\\monopanel_G173_ht_3.0.rad'])
    analysis2 = AnalysisObj(octfile, demo2.basename)
    analysis2.G173analysis(octfile, demo2.basename)

    pvscdemo = RadianceObj('PVSC_gencumsky')  
    pvscdemo.setGround('litesoil')
    epwfile = pvscdemo.getEPW(40,-105)
    metdata = pvscdemo.readEPW(epwfile)
    # sky data for index 4010 - 4028 (June 17)  
    #demo.gendaylit(metdata,4020)
    start = datetime.datetime(2000,6,17,12)
    end = datetime.datetime(2000,6,17,13)
    pvscdemo.genCumSky(r'USA_CO_Boulder.724699_TMY2.epw', start, end)
    octfile = pvscdemo.makeOct(pvscdemo.filelist + ['objects\\PVSC_4array.rad'])
    pvscdemo.analysis(octfile, pvscdemo.basename)

    pvscdemo = RadianceObj('PVSC_gendaylit')  
    pvscdemo.setGround('litesoil')
    metdata = pvscdemo.readEPW(r'USA_CO_Boulder.724699_TMY2.epw')
    # sky data for index 4010 - 4028 (June 17)  
    pvscdemo.gendaylit(metdata,4020)
    #pvscdemo.genCumSky(r'USA_CO_Boulder.724699_TMY2.epw',datetime.datetime(2000,6,17,0), datetime.datetime(2000,6,17,23))
    octfile = pvscdemo.makeOct(pvscdemo.filelist + ['objects\\PVSC_4array.rad'])
    analysis = pvscdemo.analysis(octfile, pvscdemo.basename)
    analysis.makeImage('PVSCfront.vp')
    '''